In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import statistics as stats
import csv
import random
from kuramoto import Kuramoto, plot_phase_coherence, plot_activity, plot_heatmap

In [ ]:
### Implement a method in the Kuramoto class to generate databases of colors

In [136]:
G = nx.barabasi_albert_graph(5000, 2)
sample_size = 150
# Sample from edges rather than nodes to prevent loss of information
sampled_edges = random.sample(list(G.edges), sample_size)
sample = nx.Graph()
sample.add_edges_from(sampled_edges)

In [2]:
# Sample k different graphs and probably generate a database
def sample_graphs(num_samples):
    df = pd.DataFrame(columns=['# Edges', '# Nodes', 'Min Degree', 'Max Degree', 'Diameter', 'Concentrated'])
    
    for i in range(num_samples):
        
        # sampled_edges = random.sample(list(G.edges), sample_size)
        # sample = nx.Graph()
        # sample.add_edges_from(sampled_edges)
        # nodes = random.randint(10, 15)
        G = nx.newman_watts_strogatz_graph(50, 7, 0.85)
        
        if nx.is_connected(G):
            # Number of Edges and Nodes
            edges = G.number_of_edges()
            nodes = G.number_of_nodes()

            # Min and Max degree
            degree_sequence = sorted((d for n, d in G.degree()), reverse=True)
            dmax = max(degree_sequence)
            dmin = min(degree_sequence)

            # Diameter of graph
            diam = nx.diameter(G)
            
            # Applying Kuramoto
            adj_mat = nx.to_numpy_array(G)
            natfreqs = np.repeat(2 * np.pi * 0, nodes)
            model = Kuramoto(coupling=0.75, dt=0.01, T=50, n_nodes=nodes, half_sync=False)
            sim = model.run(adj_mat)
            conc = Kuramoto.concentrated

            df.at[len(df.index)] = [edges, nodes, dmin, dmax, diam, conc]
    
    return df

In [3]:
num_samples = 10000

df = sample_graphs(num_samples)
df

,# Edges,# Nodes,Min Degree,Max Degree,Diameter,Concentrated
0,280,50,8,16,3,False
1,276,50,7,15,3,False
2,280,50,8,15,3,False
3,275,50,9,15,3,False
4,277,50,7,14,3,False
...,...,...,...,...,...,...
9995,280,50,7,15,3,False
9996,277,50,8,15,3,False
9997,280,50,7,17,3,False
9998,273,50,8,15,3,False


In [4]:
sum(df['Concentrated'])

0

In [5]:
df.to_csv('./datasets/newman50_7_0.85_10k_false')